In [4]:

import os
import numpy as np
import pandas as pd
import json
import hydra
from omegaconf import DictConfig, OmegaConf
import logging

from ERNA_GUI.core.processing_DEPRECATED import (
    load_erna_file, load_selected_file, load_erna_annots,
    preprocess_erna, extract_pulses, extract_erna_slice,
    compute_IPI_features, compute_IBI_features,
    remove_zerostim, flatten_ipi_features
)

from ERNA_GUI.io.general import (
    setup_path,
    store_ERNAfiles_results2annot,
    store_ERNAfiles_results2data,
    load_ERNAfiles_results2annot,
    load_ERNAfiles_results2data
)

from ERNA_GUI.core.utility import (
    fetch_runschema, 
    fetch_run, 
    fetch_annot,
    fetch_IBImatrix
)

from ERNA_GUI.viz.plotting import (
    plot_stimChannelLocation,
    plot_IBImatrix,
    animate_IBIavg,
    plot_IPIavg,
    animate_IPIavg,
    plot_IBIfeatures_dynamics,
    plot_IPIfeatures_dynamics,
    plot_continuous_data
)

from ERNA_GUI.api.ERNA import ERNAPlatformRequest

ERNA = load_ERNAfiles_results2data("DM2001", "argus")

PARTICIPANT = setup_path("DM2001", device= "argus")

ERNA_ = ERNAPlatformRequest(PARTICIPANT)







argus
List of parameters:
{
    "srate": 25000,
    "NPULSES_PER_BURST": 10,
    "flow_cut": 1000,
    "fhigh_cut": 2,
    "order_low": 4,
    "order_high": 4,
    "flip_polarity": true,
    "which_detrend": "linear",
    "IBI_WINDOW": {
        "25": 0.025,
        "65": 0.025,
        "130": 0.025,
        "180": 0.025
    },
    "thr_pulses": 50,
    "win_ipi_min": 2.5,
    "num_mad_outlier": 4,
    "peak_prom": 5,
    "peak_width": 12.0,
    "mindist_prop": 0.25,
    "minpeaks": 2,
    "npeaks": 8
}
Annotations loaded for participant DM2001


In [2]:
runSchema = fetch_runschema(ERNA)
print("List of possible runs:")
print(runSchema)

IPI_annot, IBI_annot = load_ERNAfiles_results2annot("DM2001","both")

List of possible runs:
{'stimScan': [0, 1, 2, 3, 4, 5, 6, 7, 21, 22, 23, 24, 25, 26, 27, 28], 'ampRamp': [8, 9, 10, 11, 29, 30, 31, 32]}


In [5]:
ERNA.keys()


dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32'])

In [11]:
for task in ["stimScan","ampRamp"]:
    print(f"Analyzing task: {task}")

    print("Fetching data...")
    task_data = fetch_run(ERNA,task=task)

    for i, run_id in enumerate(runSchema[task]): 
        # print(run)
        #run = runSchema[task][run_id]
        print(f"Extracting data and annots for run: {run_id} [task: {task}], [{i + 1}-{len(runSchema[task])}, {(i+1)/len(runSchema[task])*100:.2f}%]")
        run_data = task_data[run_id]

        IBI_annot = fetch_annot(ERNA, "IBI_annot", id= run_id)
        IPI_annot = fetch_annot(ERNA, "IPI_annot", id = run_id)

        print("Run details:")
        run_details = run_data["run_details"]
        run_details
        Burst_mode = run_details['Burst_mode'].values[0]
        
        print("Computing raw and location of recordings")
        plot_continuous_data(run_data, output_path=os.path.join(ERNA.participant_figure, 'subj-' + ERNA.participant_id + '_task-' + task + '_run-0' + str(run_id) + '_plot-continuous-argus'), show = False)
        plot_stimChannelLocation(run_data,output_path=os.path.join(ERNA.participant_figure, 'subj-' + ERNA.participant_id + '_task-' + task + '_run-0' + str(run_id) + '_plot-3DDISTAL-StimChanLocation'), show = False)
        
        print("Computing ERNA signal during the inter-burst interval (IBI)")
        plot_IBImatrix(run_data,output_path=os.path.join(ERNA.participant_figure, 'subj-' + ERNA.participant_id + '_task-' + task + '_run-0' + str(run_id) + '_plot-erna-IBI'), show= False)
        animate_IBIavg(run_data,output_path=os.path.join(ERNA.participant_figure, 'subj-' + ERNA.participant_id + '_task-' + task + '_run-0' + str(run_id) + '_animation-erna-IBI'), range_y=[-200,250], show = False)
        
        print("Computing ERNA signal during the inter-pulse interval (IPI)")
        if Burst_mode:
            plot_IPIavg(run_data, output_path=os.path.join(ERNA.participant_figure, 'subj-' + ERNA.participant_id + '_task-' + task + '_run-0' + str(run_id) + '_plot-erna-IPI'), show= False)
            animate_IPIavg(run_data,output_path=os.path.join(ERNA.participant_figure, 'subj-' + ERNA.participant_id + '_task-' + task + '_run-0' + str(run_id) + '_animation-erna-IPI'), range_y = [-100, 250], show = False)
        else:
            Warning("Burst mode is not active! Skip this step")
        
        print("Computing IBI feature dynamics over time")
        plot_IBIfeatures_dynamics(IBI_annot, ref="time", output_path=os.path.join(ERNA.participant_figure, 'subj-' + ERNA.participant_id + '_task-' + task + '_run-0' + str(run_id) + '_plot-erna-IBI-features'), show= False)
        #plot_IBIfeatures_dynamics(IBI_annot, ref="charge", stim_freq = run_data['run_details']['Freq'][0])
            
        print("Computing IPI feature dynamics over time")
        if Burst_mode:
            plot_IPIfeatures_dynamics(IPI_annot, timescale = "pulse",output_path=os.path.join(ERNA.participant_figure, 'subj-' + ERNA.participant_id + '_task-' + task + '_run-0' + str(run_id) + '_plot-erna-IPI-features_timescale-pulse'), show= False)
            plot_IPIfeatures_dynamics(IPI_annot, timescale = "burst", ref = "time",output_path=os.path.join(ERNA.participant_figure, 'subj-' + ERNA.participant_id + '_task-' + task + '_run-0' + str(run_id) + '_plot-erna-IPI-features_timescale-burst'), show= False)
        #plot_IPIfeatures_dynamics(IPI_annot, timescale = "burst", ref = "charge", stim_freq = run_data['run_details']['Freq'][0])
        else:
            Warning("Burst mode is not active! Skip this step")  

Analyzing task: stimScan
Fetching data...
Extracting data and annots for run: 0 [task: stimScan], [1-16, 6.25%]


KeyError: 'Run'